## Particle within radius or overlap calculations

In [1]:
#user-defined
import ipynb.fs.full.helper as hp

#dataframes
import pandas as pd
import h5py

#speedup
import multiprocessing as mp

#suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.TimeSeries = pd.Series 

#math
import numpy as np
import math as m
from scipy.spatial.distance import cdist


#plots
import pylab as plt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar
import seaborn as sns

#misc
import time
np.set_printoptions(suppress=True)

In [2]:
# set_number = 5
dir_save = '../../data/Sprints/AN/'
dir ='~/Documents/MyFiles/DataAnalysis/data/Sprints/Run02/'
run = 'Run02.h5'

### Calculate Expected encounters

In [3]:
df = pd.read_hdf(dir+run)

In [4]:
sync_time = df.master_time-df.master_time[0]
df.insert(1,'sync_time',sync_time)

In [5]:
# storing the odor location in a numpy array to avoid reading pandas
odor_position = np.array([[df.xsrc[i],df.ysrc[i]] for i in range (len(df.xsrc))]) 
source = np.array([[0,0]])
# distance_from_source = np.array([cdist(odor_position,source)]).flatten()

### Is Encountered odor particle within the Calculated particle radius?

The loop below computes locations of the eastwest and northsouth particles and compares with a radius (distance) to interpret if the encountered particle were close to calculated particle. The execution for the loop is 23.56 seconds for 59770 rows of data

For stationery wind sensor: The loop below takes in ith eastwest and subtracts the ith U and V elements to get the i-1 state, as the eastwest is summation of all the wind positions. 

[3 + 2 + 1] = [6]

hence ith _state = 6

_state = _state-3

_state = _state-2




In [6]:
df.keys()

Index(['master_time', 'sync_time', 'xsrc', 'ysrc', 'odor', 'trisonica_u',
       'trisonica_v', 'trisonica_pressure', 'trisonica_temperature',
       'gps_linear_x', 'gps_linear_y', 'gps_linear_z', 'imu_angular_x',
       'imu_angular_y', 'imu_angular_z', 'imu_linear_acc_x',
       'imu_linear_acc_y', 'imu_linear_acc_z', 'U', 'V', 'D', 'S2'],
      dtype='object')

In [7]:
#summation till Nth particle
dt=df.master_time[1]-df.master_time[0]
eastwest = [np.sum(df.U[j:])*dt for j in range(0,len(df))]
northsouth = [np.sum(df.V[j:])*dt for j in range(0,len(df))]

In [8]:
container=pd.DataFrame()
container['eastwest']=eastwest
container['northsouth']=northsouth

container.to_hdf(dir+'Run02_WindPath.h5', key='df', mode='w')

In [10]:
odor_presence=[]
min_distance = []
max_radius = []

In [11]:
len(eastwest)

342530

In [12]:
len(eastwest)-60000

282530

In [13]:
odor_presence=[]
points_along_streakline = []
nearest_particle= []
#for(upper, lower, step)
for i in range((len(eastwest))-1,282530 , -1):
    
    odor_pos = [odor_position[i]]  
    eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    radius = np.arange(start = 6000, stop = 0, step = -1)**0.5*0.01
    distance = cdist(odor_pos,wind_pos).flatten()   # cdist compares distance for all the points in both arrays
    nearest_particle.append(np.min(distance))
    pos = np.where(distance == np.amin(distance))

    points_along_streakline.append(wind_pos[int(pos[0]):])
    
    x = np.any(distance<=radius)             # generates a boolean values 
    if x==True:
        odor_presence.append(1)
    else:
        odor_presence.append(0)

In [17]:
for i in range((len(eastwest))-1,6000, -1):
    
    odor_pos = [odor_position[i]]  
    eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
    northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
    wind_pos = np.vstack([eastwest[-6000:],northsouth[-6000:]]).T   
    radius = np.arange(start = 6000, stop = 0, step = -1)**0.5*0.01
    distance = cdist(odor_pos,wind_pos).flatten()   # cdist compares distance for all the points in both arrays
    nearest_particle.append(np.min(distance))
    pos = np.where(distance == np.amin(distance))

    points_along_streakline.append(wind_pos[int(pos[0]):])
    
    x = np.any(distance<=radius)             # generates a boolean values 
    if x==True:
        odor_presence.append(1)
    else:
        odor_presence.append(0)

In [18]:
for i in range((len(eastwest))-1, -1, -1):
    
    odor_pos = [odor_position[i]]  
    if(i == 0):
        radius = np.zeros(1)
        wind_pos = np.array([[0,0]])
    else:
        eastwest = np.resize(np.array([eastwest-df.U[i]*dt]),(1,i)).flatten() # resize needed to avoid negative data
        northsouth = np.resize(np.array([northsouth-df.V[i]*dt]),(1,i)).flatten()
        wind_pos = np.vstack([eastwest,northsouth]).T   
        radius = np.arange(start = i, stop = 0, step = -1)**0.5*0.01
    
    distance = cdist(odor_pos,wind_pos).flatten()   # cdist compares distance for all the points in both arrays
    nearest_particle.append(np.min(distance))
    pos = np.where(distance == np.amin(distance))
    points_along_streakline.append(wind_pos[int(pos[0]):])
        
    x = np.any(distance<=radius)             # generates a boolean values 
    if x==True:
        odor_presence.append(1)
    else:
        odor_presence.append(0)

In [19]:
len(points_along_streakline)

342530

In [20]:
distance_along_streakline=[]
for i in range(0,100000):
    lengths = np.sqrt(np.sum(np.diff(np.array(points_along_streakline[i]), axis=0)**2, axis=1)) # Length between corners
    distance_along_streakline.append(np.sum(lengths))

In [21]:
for i in range(100000,200000):
    lengths = np.sqrt(np.sum(np.diff(np.array(points_along_streakline[i]), axis=0)**2, axis=1)) # Length between corners
    distance_along_streakline.append(np.sum(lengths))

In [22]:
for i in range(200000,len(points_along_streakline)):
    lengths = np.sqrt(np.sum(np.diff(np.array(points_along_streakline[i]), axis=0)**2, axis=1)) # Length between corners
    distance_along_streakline.append(np.sum(lengths))

In [23]:
len(distance_along_streakline) 

342530

In [24]:
## flip containers because above iteration is done in reverse order
df['mock'] = np.zeros(len(df))
distance_along_streakline = np.array(np.flip(distance_along_streakline)) 
odor_presence = odor_presence[::-1]
# max_radius = max_radius[::-1]

## storing in the dataframe 
df['odor_expected'] = odor_presence
df['distance_along_streakline'] =  distance_along_streakline
df['nearest_particle'] = nearest_particle
# df['distance_from_source'] = distance_from_source

In [25]:
df.to_hdf(dir+'Run02_expected.h5', key='df', mode='w')
